In [1]:
import sys
sys.path.append('../')

import json
# from instruments.swap import SwapFixedFloating_dev
from datetime import datetime, timedelta, date
import pandas as pd
import matplotlib.pyplot as plt

from utilities.time import Settings
from structures.ratecurves import DiscountCurveSimple
from index.curverateindex import IborIndex
from interface.fixedleg import FixedRateLeg
from interface.floatingleg import FloatingRateLeg


# Market instance

In [2]:
# import curva
market_dfs = pd.read_csv('../data/Market_DFs_20230621.csv', 
                            skiprows= 2,
                            on_bad_lines='skip')
df = market_dfs[market_dfs['Property'].str.contains("IR.EUR-ESTR-ON.DF")][['Property', 'Label']]
dates = []
for i in range(len(df['Property'].values)):
    str_temp = df['Property'].values[i][18:29]
    dates.append(datetime.strptime(str_temp, '%d-%b-%Y') )
    
dfs_pillars = [(dates[t]-dates[0]).days/365 for t in range(len(dates))]
dfs_market = df['Label'].values

disc_curve = DiscountCurveSimple(dfs_pillars,
                                 dfs_market)

# Fixed Leg Price

In [3]:
evaluation_date = date(2023, 6, 21)
start_date = date(2015,12,31)
fixed_schedule = [start_date]
fixed_schedule.extend([start_date + timedelta(365 *.5 *i) for i in range(1,51)])
fixed_rate = [0.04156]*(len(fixed_schedule)-1)
fixed_notional = [143857602,142099704,140995554,139343227,138170625,136487409,135100610,133235348,131633135,130088055,
                128443001,126527961,124478813,122046073,119540895,117334856,115325682,113682288,112374057,110400222,108996590,
                107427609,106277787,104934949,103826978,102318265,101011839,99241108,97427949,95062460,92643968,89488286,86306759,
                82654668,79233393,75249260,71491885,67239725,63147273,58562602,54101515,49235670,44484930,39276217,34122360,28587822,
                23186787,17327243,11581997,5627771]
leg1 = FixedRateLeg(fixed_schedule, fixed_notional, fixed_rate)

npv_value = leg1.npv(discount_curve= disc_curve,
                    evaluation_date=evaluation_date)
print(f"NPV FixedLeg: {npv_value :,.0f}")

NPV FixedLeg: 48,242,121


# Floating Leg Price

In [4]:
floating_schedule = [start_date]
floating_schedule.extend([start_date + timedelta(365 *.5 *i) for i in range(1,51)])

floating_notional = [0,0,0,0,0,0,0,0,0,0,
    128443001,126527961,124478813,122046073,119540895,117334856,115325682,113682288,112374057,110400222,108996590,
    107427609,106277787,104934949,103826978,102318265,101011839,99241108,97427949,95062460,92643968,89488286,86306759,
    82654668,79233393,75249260,71491885,67239725,63147273,58562602,54101515,49235670,44484930,39276217,34122360,28587822,
    23186787,17327243,11581997,5627771]

index_6m = IborIndex("EUR6M",
                     "calendar",
                    "6M")
spread = [0.] *(len(floating_schedule)-1)
gearing = [1.] *(len(floating_schedule)-1)

float_leg = FloatingRateLeg(floating_schedule,
                            floating_notional,
                            gearing,
                            spread,
                            index_6m)
index_6m.add_fixing(date(2022,12,29), 0.03)

npv_float = float_leg.npv(disc_curve, disc_curve, evaluation_date)

print(f"NPV FloatingLeg: {npv_float :,.0f}")

NPV FloatingLeg: 33,339,462


##### QuantLib check - FixedLeg

In [5]:
import QuantLib as ql
def curve_estr(Market_DFs):
    df = Market_DFs[Market_DFs['Property'].str.contains("IR.EUR-ESTR-ON.DF")][['Property', 'Label']]
    date_format = '%d-%b-%Y'
    dates_QuantLib = []
    for i in range(len(df['Property'].values)):
        str_temp = df['Property'].values[i][18:29]
        datetime_obj = datetime.strptime(str_temp, date_format)
        dates_QuantLib.append(ql.Date.from_date(datetime_obj))

    DFs = df['Label'].values
    yieldTermStructure = ql.DiscountCurve(dates_QuantLib, DFs, ql.Actual360(), ql.TARGET())
    yieldTermStructure.enableExtrapolation()
    return yieldTermStructure

disc_curve_ql = curve_estr(market_dfs)
start_date_ql = ql.Date(31,12,2015)
end_date_ql = ql.Date(31,12,2040)
fixed_tenor_ql = ql.Period('6M')
fixed_schedule_ql = ql.MakeSchedule(start_date_ql, end_date_ql, fixed_tenor_ql)
leg_ql = ql.FixedRateLeg(fixed_schedule_ql, ql.Actual360(), fixed_notional, fixed_rate)
ql.Settings.instance().evaluationDate = ql.Date(21,6,2023)

print(f"NPV FixedLeg TQ: {npv_value :,.0f}")
print(f"NPV FixedLeg QL: {ql.CashFlows.npv(leg_ql, ql.YieldTermStructureHandle(disc_curve_ql), True) :,.0f}")

NPV FixedLeg TQ: 48,242,121
NPV FixedLeg QL: 48,926,821


##### QuantLib check - FloatingLeg

In [6]:
start_date = ql.Date(31,12,2015)
end_date = ql.Date(31,12,2040)

floating_tenor = ql.Period('6M')
floating_schedule_ql = ql.MakeSchedule(start_date, end_date, floating_tenor)
floatDayCount = ql.Actual360()
index_eur6m_ql = ql.Euribor6M(ql.YieldTermStructureHandle(disc_curve_ql))
index_eur6m_ql.addFixing(ql.Date(29,12,2022), 0.03)

leg2_ql = ql.IborLeg(floating_notional, floating_schedule_ql, index_eur6m_ql)

print(f"NPV FloatingLeg TQ: {npv_float :,.0f}")
print(f"NPV FloatingLeg QL: {ql.CashFlows.npv(leg2_ql, ql.YieldTermStructureHandle(disc_curve_ql), True) :,.0f}")

NPV FloatingLeg TQ: 33,339,462
NPV FloatingLeg QL: 33,376,616


# Display Leg information

In [7]:
flows = float_leg.leg_flows()
# period_start_date = []
# period_end_date = []
# for i in range(len(flows)):
#     period_start_date.append(float_leg.schedule[i])
#     period_end_date.append(float_leg.schedule[i+1])
# amounts = [flows[t].amount for t in range(len(flows))]
# leg_overview = {}
# leg_overview['period_start_date'] = period_start_date
# leg_overview['period_end_date'] = period_end_date
# leg_overview['notional'] = float_leg.notionals

In [8]:
i = 21
cf = flows[i]

In [9]:
# def display_anag(disc_curve,
#                  term_structure,
#                 evaluation_date):
#     flows = float_leg.leg_flows()
#     leg_display = pd.DataFrame()
#     for i in range(len(flows)):
#         coupon_flow = flows[i].display(disc_curve,
#                                         term_structure,
#                                         evaluation_date)
#         leg_display = pd.concat([leg_display, coupon_flow], axis = 0)
#     return leg_display

In [10]:
float_leg.display_flows(disc_curve,
                 disc_curve,
                evaluation_date)

,start_period,end_period,payment_date,notional,fixing_date,fixing_days,index,fixing,accrual,in_arrears,gearing,spread,day_counter,amount,discount_factor,coupon_pv,is_expired
0,2015-12-31,2016-06-30,2016-06-30,0,2015-12-31,None,EUR6M,0,0.49863,False,1.0,0.0,None,0,0,0,True
0,2016-06-30,2016-12-30,2016-12-30,0,2016-06-30,None,EUR6M,0,0.50137,False,1.0,0.0,None,0,0,0,True
0,2016-12-30,2017-06-30,2017-06-30,0,2016-12-30,None,EUR6M,0,0.49863,False,1.0,0.0,None,0,0,0,True
0,2017-06-30,2017-12-30,2017-12-30,0,2017-06-30,None,EUR6M,0,0.50137,False,1.0,0.0,None,0,0,0,True
0,2017-12-30,2018-06-30,2018-06-30,0,2017-12-30,None,EUR6M,0,0.49863,False,1.0,0.0,None,0,0,0,True
0,2018-06-30,2018-12-30,2018-12-30,0,2018-06-30,None,EUR6M,0,0.50137,False,1.0,0.0,None,0,0,0,True
0,2018-12-30,2019-06-30,2019-06-30,0,2018-12-30,None,EUR6M,0,0.49863,False,1.0,0.0,None,0,0,0,True
0,2019-06-30,2019-12-30,2019-12-30,0,2019-06-30,None,EUR6M,0,0.50137,False,1.0,0.0,None,0,0,0,True
0,2019-12-30,2020-06-29,2020-06-29,0,2019-12-30,None,EUR6M,0,0.49863,False,1.0,0.0,None,0,0,0,True
0,2020-06-29,2020-12-29,2020-12-29,0,2020-06-29,None,EUR6M,0,0.50137,False,1.0,0.0,None,0,0,0,True
